In [1]:
import os
import numpy as np
import torch
import gc
from tqdm import tqdm
from torchvision.ops import box_convert
from groundingdino.util.inference import load_model, load_image, predict, annotate
from segment_anything import sam_model_registry, SamPredictor
import openpyxl
from contextlib import contextmanager

# 忽略特定警告
import warnings
warnings.filterwarnings("ignore", category=FutureWarning, message=".*device.*")
warnings.filterwarnings("ignore", category=UserWarning, message=".*requires_grad.*")

# 使用 GPU 设备（如果可用）
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# 加载 GroundingDINO 模型
cfg_path = r'D://code//CV//GroundingDINO//groundingdino//config//GroundingDINO_SwinT_OGC.py'
weight_path = r'd://code//CV//GroundingDINO//weight//groundingdino_swint_ogc.pth'
gdino_model = load_model(cfg_path, weight_path)
gdino_model.to(device)
BOX_THRESHOLD = 0.35
TEXT_THRESHOLD = 0.35

# 加载 Segment Anything 模型
sam_checkpoint = r"D://aigc//ComfyUI_windows_portable//ComfyUI//models//sams//sam_vit_h_4b8939.pth"
model_type = "vit_h"
sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device)


@contextmanager
def torch_gc():
    try:
        yield
    finally:
        gc.collect()
        torch.cuda.empty_cache()


def process_images_in_folder_structure(input_folder, text_prompt):
    total_files = 0
    processed_files = 0
    skipped_files = 0

    second_level_folders = [f.path for f in os.scandir(input_folder) if f.is_dir()]

    for second_folder in second_level_folders:
        print(f"Processing second level folder: {second_folder}")

        third_level_folders = [f.path for f in os.scandir(second_folder) if f.is_dir()]

        for third_folder in third_level_folders:
            print(f"Processing third level folder: {third_folder}")

            output_folder = os.path.join(third_folder, 'grounding_output')
            if not os.path.exists(output_folder):
                os.makedirs(output_folder)

            image_files = [f for f in os.listdir(third_folder) if f.lower().endswith((".jpg", ".png"))]
            total_files += len(image_files)

            for filename in tqdm(image_files, desc=f"Processing {third_folder}"):
                ip = os.path.join(third_folder, filename)
                base_name = os.path.basename(ip).split('.')[0]
                output_filename = f"{base_name}.jpg"
                output_path = os.path.join(output_folder, output_filename)

                # 检查输出文件是否已存在
                if os.path.exists(output_path):
                    print(f"Skipping {filename} as it has already been processed.")
                    skipped_files += 1
                    continue

                try:
                    with torch_gc():
                        image_source, boxes = get_box_from_gdino_with_text_prompt(ip, text_prompt)
                        if image_source is not None and boxes is not None:
                            seg_with_sam_with_box_prompt(image_source, boxes, text_prompt, output_folder, ip)
                            processed_files += 1
                except Exception as e:
                    print(f"处理文件 {filename} 时出错: {e}")
                finally:
                    gc.collect()
                    torch.cuda.empty_cache()

    print(f"总文件数: {total_files}")
    print(f"成功处理的文件数: {processed_files}")
    print(f"跳过的文件数: {skipped_files}")


def get_box_from_gdino_with_text_prompt(ip, text_prompt):
    try:
        image_source, image = load_image(ip)
        if image_source is None or image is None:
            print(f"无法加载图像: {ip}")
            return None, None

        h, w = image_source.shape[:2]
        boxes, logits, phrases = predict(
            model=gdino_model,
            image=image,
            caption=text_prompt,
            box_threshold=BOX_THRESHOLD,
            text_threshold=TEXT_THRESHOLD,
            device=device
        )

        # 尽早释放内存
        del image
        gc.collect()

        boxes = boxes.to(device)
        scale_tensor = torch.tensor([w, h, w, h], dtype=boxes.dtype, device=device)
        boxes = boxes * scale_tensor
        xyxy = box_convert(boxes=boxes, in_fmt="cxcywh", out_fmt="xyxy").cpu().numpy()

        del logits, phrases, boxes, scale_tensor
        gc.collect()
        torch.cuda.empty_cache()
        return image_source, xyxy
    except Exception as e:
        print(f"加载图像 {ip} 时出错: {e}")
        return None, None


def seg_with_sam_with_box_prompt(image, boxes, text_prompt, output_folder, ip):
    predictor = SamPredictor(sam)
    predictor.set_image(image)
    merged_mask = np.zeros((image.shape[0], image.shape[1]), dtype=bool)
    base_name = os.path.basename(ip).split('.')[0]

    for idx, box in enumerate(boxes):
        input_box = box.astype(int)
        masks, _, _ = predictor.predict(
            point_coords=None,
            point_labels=None,
            box=input_box[None, :],
            multimask_output=False,
        )
        if masks is not None and len(masks) > 0:
            mask = masks[0]
            merged_mask = np.logical_or(merged_mask, mask)
        del masks
        gc.collect()

    if np.any(merged_mask):
        output_filename = f"{base_name}.jpg"
        output_path = os.path.join(output_folder, output_filename)
        save_masked_image_using_pillow(image, merged_mask, output_path, boxes)

        vertex_coords = [box for box in boxes]

        excel_file = os.path.join(output_folder, 'grounding_results.xlsx')
        update_excel(excel_file, base_name, boxes)


def save_masked_image_using_pillow(image_np, mask_np, output_path, boxes):
    from PIL import Image
    image = Image.fromarray(image_np)
    masked_image = Image.new('RGBA', image.size)
    masked_image.paste(image, (0, 0))

    # 将mask颜色改为蓝色（这里是RGB值）
    blue_color = (0, 0, 255)
    blue_mask_image = Image.new('L', image.size, 0)
    blue_mask_image.putdata((mask_np * 255).astype('uint8').flatten())
    blue_mask_image = blue_mask_image.convert('RGBA')
    blue_mask_image_data = np.array(blue_mask_image)
    blue_mask_image_data[blue_mask_image_data[:, :, 3] > 0] = [*blue_color, 255]
    blue_mask_image = Image.fromarray(blue_mask_image_data)

    masked_image = Image.alpha_composite(masked_image, blue_mask_image)

    if masked_image.mode == 'RGBA':
        masked_image = masked_image.convert('RGB')

    masked_image.save(output_path)


def update_excel(excel_file, base_name, boxes):
    if not os.path.exists(excel_file):
        workbook = openpyxl.Workbook()
        worksheet = workbook.active
        worksheet.append(['Image Name', 'x1', 'y1', 'x2', 'y2', 'x3', 'y3', 'x4', 'y4'])
        new_rows = []
        for i, box in enumerate(boxes):
            new_rows.append([
                f"{base_name}_{i + 1}",
                box[0], box[1],
                box[2], box[1],
                box[2], box[3],
                box[0], box[3]
            ])
        for row in new_rows:
            worksheet.append(row)
        workbook.save(excel_file)
    else:
        workbook = openpyxl.load_workbook(excel_file)
        worksheet = workbook.active
        new_rows = []
        for i, box in enumerate(boxes):
            new_rows.append([
                f"{base_name}_{i + 1}",
                box[0], box[1],
                box[2], box[1],
                box[2], box[3],
                box[0], box[3]
            ])
        for row in new_rows:
            worksheet.append(row)
        workbook.save(excel_file)


if __name__ == '__main__':
    input_folder = "D://code//data//Lv2期结论//test"
    text = 'person,shoes'
    process_images_in_folder_structure(input_folder, text)
    print('完结完结完结')


    

KeyboardInterrupt: 

In [2]:
import os
import numpy as np
import torch
import gc
from tqdm import tqdm
from torchvision.ops import box_convert
from groundingdino.util.inference import load_model, load_image, predict, annotate
from segment_anything import sam_model_registry, SamPredictor
import openpyxl
from contextlib import contextmanager

# 忽略特定警告
import warnings
warnings.filterwarnings("ignore", category=FutureWarning, message=".*device.*")
warnings.filterwarnings("ignore", category=UserWarning, message=".*requires_grad.*")

# 使用 GPU 设备（如果可用）
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# 加载 GroundingDINO 模型
cfg_path = r'D://code//CV//GroundingDINO//groundingdino//config//GroundingDINO_SwinT_OGC.py'
weight_path = r'd://code//CV//GroundingDINO//weight//groundingdino_swint_ogc.pth'
gdino_model = load_model(cfg_path, weight_path)
gdino_model.to(device)
BOX_THRESHOLD = 0.35
TEXT_THRESHOLD = 0.35

# 加载 Segment Anything 模型
sam_checkpoint = r"D://aigc//ComfyUI_windows_portable//ComfyUI//models//sams//sam_vit_h_4b8939.pth"
model_type = "vit_h"
sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device)


@contextmanager
def torch_gc():
    try:
        yield
    finally:
        gc.collect()
        torch.cuda.empty_cache()


def process_images_in_folder_structure(input_folder, text_prompt):
    total_files = 0
    processed_files = 0
    skipped_files = 0

    second_level_folders = [f.path for f in os.scandir(input_folder) if f.is_dir()]

    for second_folder in second_level_folders:
        print(f"Processing second level folder: {second_folder}")

        third_level_folders = [f.path for f in os.scandir(second_folder) if f.is_dir()]

        for third_folder in third_level_folders:
            print(f"Processing third level folder: {third_folder}")

            output_folder = os.path.join(third_folder, 'grounding_output')
            if not os.path.exists(output_folder):
                os.makedirs(output_folder)

            image_files = [f for f in os.listdir(third_folder) if f.lower().endswith((".jpg", ".png"))]
            total_files += len(image_files)

            for filename in tqdm(image_files, desc=f"Processing {third_folder}"):
                ip = os.path.join(third_folder, filename)
                base_name = os.path.basename(ip).split('.')[0]
                output_filename = f"{base_name}.jpg"
                output_path = os.path.join(output_folder, output_filename)

                # 检查输出文件是否已存在
                if os.path.exists(output_path):
                    print(f"Skipping {filename} as it has already been processed.")
                    skipped_files += 1
                    continue

                try:
                    with torch_gc():
                        image_source, boxes = get_box_from_gdino_with_text_prompt(ip, text_prompt)
                        if image_source is not None and boxes is not None:
                            seg_with_sam_with_box_prompt(image_source, boxes, text_prompt, output_folder, ip)
                            processed_files += 1
                except Exception as e:
                    print(f"处理文件 {filename} 时出错: {e}")
                finally:
                    gc.collect()
                    torch.cuda.empty_cache()

    print(f"总文件数: {total_files}")
    print(f"成功处理的文件数: {processed_files}")
    print(f"跳过的文件数: {skipped_files}")


def get_box_from_gdino_with_text_prompt(ip, text_prompt):
    try:
        image_source, image = load_image(ip)
        if image_source is None or image is None:
            print(f"无法加载图像: {ip}")
            return None, None

        h, w = image_source.shape[:2]
        boxes, logits, phrases = predict(
            model=gdino_model,
            image=image,
            caption=text_prompt,
            box_threshold=BOX_THRESHOLD,
            text_threshold=TEXT_THRESHOLD,
            device=device
        )

        # 尽早释放内存
        del image
        gc.collect()

        boxes = boxes.to(device)
        scale_tensor = torch.tensor([w, h, w, h], dtype=boxes.dtype, device=device)
        boxes = boxes * scale_tensor
        xyxy = box_convert(boxes=boxes, in_fmt="cxcywh", out_fmt="xyxy").cpu().numpy()

        del logits, phrases, boxes, scale_tensor
        gc.collect()
        torch.cuda.empty_cache()
        return image_source, xyxy
    except Exception as e:
        print(f"加载图像 {ip} 时出错: {e}")
        return None, None


def seg_with_sam_with_box_prompt(image, boxes, text_prompt, output_folder, ip):
    predictor = SamPredictor(sam)
    predictor.set_image(image)
    merged_mask = None
    base_name = os.path.basename(ip).split('.')[0]

    for idx, box in enumerate(boxes):
        input_box = box.astype(int)
        masks, _, _ = predictor.predict(
            point_coords=None,
            point_labels=None,
            box=input_box[None, :],
            multimask_output=False,
        )
        if masks is not None and len(masks) > 0:
            if merged_mask is None:
                merged_mask = masks[0]
            else:
                merged_mask = np.logical_or(merged_mask, masks[0])
        # 及时释放内存
        del masks
        gc.collect()

    if merged_mask is not None:
        merged_mask = np.where(merged_mask > 0, 1, 0)
        if np.any(merged_mask):
            output_filename = f"{base_name}.jpg"
            output_path = os.path.join(output_folder, output_filename)
            save_masked_image_using_pillow(image, merged_mask, output_path, boxes)

            vertex_coords = [box for box in boxes]

            excel_file = os.path.join(output_folder, 'grounding_results.xlsx')
            update_excel(excel_file, base_name, boxes)


def save_masked_image_using_pillow(image_np, mask_np, output_path, boxes):
    from PIL import Image, ImageDraw
    
    # 将numpy数组转换为PIL图像
    image = Image.fromarray(image_np)
    
    # 创建一个与原始图像相同大小的RGBA图像
    result_image = Image.new('RGBA', image.size)
    result_image.paste(image, (0, 0))  # 首先粘贴原始图像
    
    # 创建蓝色遮罩
    blue_mask = Image.new('RGBA', image.size, (0, 0, 255, 128))  # 半透明蓝色
    mask_image = Image.fromarray((mask_np * 255).astype('uint8'))
    
    # 将蓝色遮罩应用到目标区域
    result_image.paste(blue_mask, (0, 0), mask_image)
    
    # # 绘制边界框
    # draw = ImageDraw.Draw(result_image)
    # for box in boxes:
    #     draw.rectangle(box.tolist(), outline='red', width=2)
    
    # 如果需要保存为JPG格式，转换为RGB模式
    if output_path.lower().endswith('.jpg'):
        result_image = result_image.convert('RGB')
    
    # 保存结果
    result_image.save(output_path)


def update_excel(excel_file, base_name, boxes):
    if not os.path.exists(excel_file):
        workbook = openpyxl.Workbook()
        worksheet = workbook.active
        worksheet.append(['Image Name', 'x1', 'y1', 'x2', 'y2', 'x3', 'y3', 'x4', 'y4'])
        new_rows = []
        for i, box in enumerate(boxes):
            new_rows.append([
                f"{base_name}_{i + 1}",
                box[0], box[1],
                box[2], box[1],
                box[2], box[3],
                box[0], box[3]
            ])
        for row in new_rows:
            worksheet.append(row)
        workbook.save(excel_file)
    else:
        workbook = openpyxl.load_workbook(excel_file)
        worksheet = workbook.active
        new_rows = []
        for i, box in enumerate(boxes):
            new_rows.append([
                f"{base_name}_{i + 1}",
                box[0], box[1],
                box[2], box[1],
                box[2], box[3],
                box[0], box[3]
            ])
        for row in new_rows:
            worksheet.append(row)
        workbook.save(excel_file)


if __name__ == '__main__':
    input_folder = "D://code//data//Lv2期结论//test"
    text = 'person,shoes'
    process_images_in_folder_structure(input_folder, text)
    print('完结完结完结')





Using device: cuda
final text_encoder_type: bert-base-uncased


c:\ProgramData\miniconda3\lib\site-packages\transformers\tokenization_utils_base.py:1602: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Processing second level folder: D://code//data//Lv2期结论//test\testtest
Processing third level folder: D://code//data//Lv2期结论//test\testtest\1047


Processing D://code//data//Lv2期结论//test\testtest\1047:   0%|          | 0/35 [00:00<?, ?it/s]

Skipping 63db832cF97eff165_b8c23b3d7354a1d4.jpg as it has already been processed.
Skipping 65d70348Fd9d3b730_1f93b1aac6b19013.jpg as it has already been processed.
Skipping 65d832a0F54b9d69b_fff2a9f29b14ab5b.jpg as it has already been processed.


...

Processing third level folder: D://code//data//Lv2期结论//test\testtest\9435


Processing D://code//data//Lv2期结论//test\testtest\9435: 100%|██████████| 178/178 [03:08<00:00,  1.06s/it]


Processing third level folder: D://code//data//Lv2期结论//test\testtest\9775


Processing D://code//data//Lv2期结论//test\testtest\9775: 100%|██████████| 33/33 [00:45<00:00,  1.38s/it]

总文件数: 1238
成功处理的文件数: 1119
跳过的文件数: 119
完结完结完结


In [ ]:
filter_layer_cases = [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]]

filter_suffix_list = ['filter_1.0_2.0','filter_3.0_4.0','filter_5.0_6.0']


x_list = ['9783','6913','6912','6911','6910','6909','6908','12066']
y_list = ['txt', 'price']